In [1]:
import pandas as pd

# Memuat dataset FAQ
data = pd.read_csv('data.csv')

# Menampilkan data
print(data.head())

                                          Pertanyaan  \
0               Apa fungsi chatbot thumbnail ini #1?   
1     Bagaimana cara kerja chatbot thumbnail ini #1?   
2  Apakah chatbot ini bisa mengenali topik berita...   
3  Apakah saya bisa langsung menggunakan chatbot ...   
4          Apakah chatbot ini bisa dipakai di HP #1?   

                                             Jawaban  
0  Chatbot ini berfungsi untuk membuat kata-kata ...  
1  Cukup masukkan judul berita ke kolom input, ch...  
2  Ya, chatbot ini dilengkapi sistem pakar yang b...  
3  Tentu, Anda bisa langsung memasukkan judul ber...  
4  Bisa. Anda cukup membuka link aplikasinya di b...  


In [2]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Memuat dataset dari file Excel
data = pd.read_csv('data.csv')
questions = data['Pertanyaan'].tolist()
answers = data['Jawaban'].tolist()

fallback_response = "Maaf, saya tidak bisa menjawab pertanyaan itu."

def get_answer(user_question):
    similarity_threshold = 60  # Lowered threshold for partial matching
    
    # Use partial_ratio to allow partial matching
    best_match, best_score = process.extractOne(user_question, questions, scorer=fuzz.partial_ratio)
    
    if best_score >= similarity_threshold:
        answer_index = questions.index(best_match)
        return answers[answer_index]
    else:
        return fallback_response

while True:
    user_question = input("Tanyakan sesuatu (ketik 'exit' untuk keluar): ")
    
    if user_question.lower() == "exit":
        break
    
    response = get_answer(user_question)
    print("Chatbot:", response)

In [3]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from fuzzywuzzy import fuzz
from sklearn.feature_extraction.text import TfidfVectorizer
import gradio as gr

# Load NLTK stopwords
try:
    stop_words = list(stopwords.words('indonesian'))
except LookupError:
    nltk.download('stopwords')
    stop_words = list(stopwords.words('indonesian'))

# Memuat dataset
try:
    data = pd.read_csv('data.csv')
    questions = data['Pertanyaan'].tolist()
    answers = data['Jawaban'].tolist()
except Exception as e:
    print("Gagal memuat data:", e)
    questions, answers = [], []

fallback_response = "Maaf, saya tidak bisa menjawab pertanyaan itu."

# Fungsi menjawab pertanyaan berbasis fuzzy partial_ratio
def get_answer(user_question):
    similarity_threshold = 50
    best_score = 0
    best_index = -1

    for i, question in enumerate(questions):
        score = fuzz.partial_ratio(user_question.lower(), question.lower())
        if score > best_score:
            best_score = score
            best_index = i

    if best_score >= similarity_threshold and best_index != -1:
        return answers[best_index]
    else:
        return fallback_response

# Fungsi ekstraksi keyword sederhana
def extract_keywords_simple(text, max_words=7):
    words = re.findall(r'\b\w+\b', text.lower())
    blacklist = {'dear', 'hai', 'halo', 'mohon', 'agar', 'harus', 'akan', 'terlalu'}
    filtered_set = set(stop_words).union(blacklist)
    filtered = [w for w in words if w not in filtered_set]
    return " ".join(filtered[:max_words])

# Fungsi hanya untuk mendapatkan tag expert
def get_expert_tags(title):
    title_lower = title.lower()
    expert_tags = []

    rules = [
        (["bencana", "gempa", "banjir", "longsor", "tsunami", "erupsi"], "Urgent"),
        (["ekonomi", "inflasi", "rupiah", "investasi", "saham", "krisis finansial"], "Update Ekonomi"),
        (["politik", "pemilu", "kampanye", "kpu", "partai", "caleg", "pilpres"], "Pemilu dan Politik"),
        (["krisis", "korupsi", "skandal", "suap", "kecurangan"], "Isu Politik & Hukum"),
        (["kesehatan", "covid", "virus", "vaksin", "rs", "dokter", "kesehatan mental"], "Info Kesehatan"),
        (["ai", "kecerdasan buatan", "teknologi", "startup", "gadget", "robot", "iot"], "Teknologi & AI"),
        (["transportasi", "kecelakaan", "pesawat", "kereta", "lalu lintas", "macet"], "Berita Transportasi"),
        (["perbankan", "bca", "bni", "biaya admin", "kartu kredit", "pinjaman"], "Keuangan & Bank"),
        (["pendidikan", "mahasiswa", "sekolah", "guru", "beasiswa", "ujian"], "Pendidikan"),
        (["olahraga", "liga", "pemain", "sepak bola", "basket", "timnas"], "Olahraga"),
        (["artis", "film", "konser", "drama", "idol", "gosip", "selebriti"], "Hiburan & Seleb"),
        (["budaya", "tradisi", "seni", "adat", "agama", "kearifan lokal"], "Sosial Budaya"),
        (["lingkungan", "polusi", "cuaca", "iklim", "hutan", "banjir", "sampah"], "Lingkungan Hidup"),
        (["kriminal", "penipuan", "narkoba", "pembunuhan", "perampokan", "tersangka"], "Kriminal & Hukum"),
        (["sosial", "kemiskinan", "bantuan", "masyarakat", "pengungsi"], "Isu Sosial"),
    ]

    for keywords_list, tag in rules:
        if any(word in title_lower for word in keywords_list):
            expert_tags.append(tag)

    return expert_tags

# Fungsi generate thumbnail
def generate_thumbnail_text(title):
    if not title.strip():
        return "⚠️ Judul kosong, mohon masukkan judul yang valid."
    keywords = extract_keywords_simple(title)
    if not keywords:
        return "⚠️ Tidak ada kata kunci yang bisa diekstrak."

    expert_tags = get_expert_tags(title)
    tags_text = f"\n📌 Tags: {', '.join(expert_tags)}" if expert_tags else ""

    return f"🖼️ Thumbnail: {keywords}{tags_text}"

# Fungsi antarmuka Gradio
def chatbot_interface(history, user_input):
    user_input = user_input.strip()
    history = history or []

    if not user_input:
        response = "⚠️ Masukkan judul atau pertanyaan terlebih dahulu."
    else:
        answer = get_answer(user_input)
        if answer == fallback_response:
            if len(user_input.split()) > 4:
                response = generate_thumbnail_text(user_input)
            else:
                response = fallback_response
        else:
            response = answer

    history.append((user_input, response))
    return history, ""

# Inisialisasi chat pertama
def initialize_chat():
    return [(None, "Halo! Kirim pertanyaan untuk dijawab atau judul berita untuk dibuatkan thumbnail.")]

# Buat UI Gradio
with gr.Blocks() as iface:
    gr.Markdown("<h2 style='text-align: center;'>🤖 Chatbot Thumbnail & QA</h2>")
    chatbot = gr.Chatbot(value=initialize_chat(), label="Chatbot")

    with gr.Row():
        with gr.Column(scale=12):
            user_input = gr.Textbox(show_label=False, placeholder="Masukkan judul atau pertanyaan...", lines=1)
        with gr.Column(scale=1):
            submit_btn = gr.Button("Kirim")

    submit_btn.click(fn=chatbot_interface, inputs=[chatbot, user_input], outputs=[chatbot, user_input])
    user_input.submit(fn=chatbot_interface, inputs=[chatbot, user_input], outputs=[chatbot, user_input])

iface.launch(share=True)


c:\Users\lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\gradio\components\chatbot.py:225: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
